In [1]:
from keras.models import Model, Sequential
from keras import models
import cv2
import numpy as np
from glob import glob
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.applications import xception
import math
xception_bf = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
categories = ['fire', 'flood', 'normal']

In [2]:
model_path = '../output/model/model_07_26_2022_17_41_18.h5'
model = models.load_model(model_path)
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [3]:
# #masking function
def create_mask_for_plant(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_hsv = np.array([0,0,250])
    upper_hsv = np.array([250,255,255])
    
    mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return mask

#image segmentation function
def segment_image(image):
    mask = create_mask_for_plant(image)
    output = cv2.bitwise_and(image, image, mask = mask)
    return output/255

#sharpen the image
def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

# function to get an image
def read_img(filepath, size):
    #print(f"filepath passed is: {filepath} and full path is: {path}")
    img = image.load_img(filepath, target_size=size)
    #convert image to array
    img = image.img_to_array(img)
    return img

In [4]:
INPUT_SIZE=255
def load_image(img_path, show=False):
    img = read_img(img_path,(INPUT_SIZE,INPUT_SIZE))
    #masking and segmentation
    image_segmented = segment_image(img)
    #sharpen
    image_sharpen = sharpen_image(image_segmented)
    output = xception.preprocess_input(np.expand_dims(image_sharpen.copy(), axis=0))
    if show:
        plt.imshow(img/255)                           
        plt.axis('off')
        plt.show()

    return output

In [5]:
from time import sleep
def decision_algorithm(frame_path, batch_size=10):
    predictions=[]
    frame_list = []
    #print(f"frame_list is of type: {type(frame_list)}")
    paths = [os.path.join(frame_path,file_name) for file_name in os.listdir(os.path.join(frame_path))]
    number_of_images = len(paths)
    current_index = 0
    count = 0;
    new_batch,current_index = get_next_batch(paths,current_index,number_of_images,10)
    while new_batch:
        images=fetch_image(new_batch)
        processed_image = process_image(images)
        prediction = predict(processed_image)
        print(f"prediction {count} is: {prediction}")
        predictions.append(prediction)
        sleep(0.1)
        new_batch,current_index = get_next_batch(paths,current_index,number_of_images,10)
        count=count+1
    decision_value = voting_system(predictions)
    print(f"decision is {categories[decision_value]}")
    decide(decision_value)

In [6]:
THRESHOLD = 50
NUMBER_OF_CLASSES = 3
def voting_system(prediction_list):
    decision = 2
    length = len(prediction_list)
    votes=[0]*NUMBER_OF_CLASSES
    percentages=[0]*NUMBER_OF_CLASSES
    total_count=0
    for prediction in prediction_list:
        votes[prediction]+=1
        total_count+=1
        
    for index,percentage in enumerate(percentages):
        percentages[index] = votes[index]/total_count*100
        print(f"percentage for {categories[index]} is: {percentages[index]}")
        if( (index == 0 or index ==1 ) and percentages[index]) > THRESHOLD:
            decision = index
    return decision
            
    

In [7]:
def get_next_batch(paths,current,length,batch_size=10):
    end=current+batch_size
    print(f"current is: {current} and end is: {end}")
    if end < length:
        batch = paths[current:end]
        return (batch,end)
    else:
        batch = paths[current:length]
        return (batch,length)
         
    

In [8]:
def fetch_image(image_paths):
    frames = []
    for img_path in image_paths:
        img = read_img(img_path,(INPUT_SIZE,INPUT_SIZE))  
        frames.append(img)
    return frames

In [9]:
def process_image(image_array):
    frame_list=[]
    for img in image_array:
        image_segmented = segment_image(img)
        #sharpen
        image_sharpen = sharpen_image(image_segmented)
        output = xception.preprocess_input(np.expand_dims(image_sharpen.copy(), axis=0))
        frame_list.append(output)
    return frame_list

In [10]:
import statistics
def predict(process_image):
    predictions=[]
    for frame in process_image:
        feature = xception_bf.predict(frame, batch_size=32)
        res = model.predict(feature)
        classes = np.argmax(res, axis = -1)
        predictions.append(classes)
        names = [categories[i] for i in classes]
        print(names) 
    predictions=[i for s in predictions for i in s]
    print(predictions)
    final_prediction = statistics.mode(predictions)
    print(f"final prediction is: {final_prediction}")
    return final_prediction

In [11]:
def decide(voting_output):
    if voting_output == 0 or voting_output == 1:
        print("Extreme condition: Find alternative route")
    else:
        print("Normal condition: Proceed")

In [12]:
frame_dir = "../input/frames/frames_two"
decision_algorithm(frame_dir)

current is: 0 and end is: 10
['flood']
['fire']
['fire']
['normal']
['normal']
['normal']
['fire']
['fire']
['normal']
['normal']
[1, 0, 0, 2, 2, 2, 0, 0, 2, 2]
final prediction is: 2
prediction 0 is: 2
current is: 10 and end is: 20
['flood']
['fire']
['fire']
['fire']
['fire']
['fire']
['fire']
['fire']
['fire']
['fire']
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
final prediction is: 0
prediction 1 is: 0
current is: 20 and end is: 30
['fire']
['fire']
['fire']
['fire']
['fire']
['fire']
['fire']
['fire']
['fire']
['fire']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
final prediction is: 0
prediction 2 is: 0
current is: 30 and end is: 40
['fire']
['fire']
['fire']
['fire']
['flood']
['fire']
['fire']
['fire']
['fire']
['fire']
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
final prediction is: 0
prediction 3 is: 0
current is: 40 and end is: 50
['fire']
['fire']
['fire']
['fire']
['fire']
['flood']
['flood']
['flood']
[0, 0, 0, 0, 0, 1, 1, 1]
final prediction is: 0
prediction 4 is: 0
current is: 48 and end is: 58
percentage for